In [1]:
import json
import logging
import os
import shutil
from zipfile import ZipFile
import requests
from io import BytesIO
import mimetypes
from time import sleep
from PIL import Image
from typing import List, Tuple
from yaml import dump

log = logging.getLogger(__name__)

In [2]:
class CVAT_API:
    def __init__(self, host, https=False):
        if host.startswith("https://"):
            https = True
        if host.startswith("http://") or host.startswith("https://"):
            host = host.replace("http://", "")
            host = host.replace("https://", "")
        scheme = "https" if https else "http"
        self.base = f"{scheme}://{host}/api/"

    @property
    def tasks(self):
        return self.base + "tasks"

    def tasks_page(self, page_id):
        return self.tasks + "?page={}".format(page_id)

    def tasks_id(self, task_id):
        return self.tasks + "/{}".format(task_id)

    def tasks_id_data(self, task_id):
        return self.tasks_id(task_id) + "/data"

    def tasks_id_dataset(self, task_id, name, fileformat):
        # format fixed to COCO 1.0
        return self.tasks_id(task_id) + "/dataset?filename{}&format={}".format(
            name, fileformat
        )

    def tasks_id_frame_id(self, task_id, frame_id, quality):
        return self.tasks_id(task_id) + "/data?type=frame&number={}&quality={}".format(
            frame_id, quality
        )

    def tasks_id_status(self, task_id):
        return self.tasks_id(task_id) + "/status"

    def tasks_id_annotations_format(self, task_id, fileformat):
        return self.tasks_id(task_id) + "/annotations?format={}".format(fileformat)

    def tasks_id_annotations_filename(self, task_id, name, fileformat):
        return self.tasks_id(task_id) + "/annotations?format={}&filename={}".format(
            fileformat, name
        )

    def get_file_format(self):
        return self.base + "server/annotation/formats"

    @property
    def login(self):
        return self.base + "auth/login"


In [3]:
class CVAT:
    def __init__(self, username: str, password: str, host: str, dump_dir: str):
        self.session = requests.Session()
        self.api = CVAT_API(host=host)
        self.login([username, password])
        self.dump_dir = dump_dir

    def tasks_list(self, use_json_output=True, **kwargs):
        """List all tasks in either basic or JSON format."""
        url = self.api.tasks
        response = self.session.get(url)
        response.raise_for_status()
        output = []
        page = 1
        json_data_list = []
        while True:
            response_json = response.json()
            output += response_json["results"]
            for r in response_json["results"]:
                if use_json_output:
                    json_data_list.append(r)
                else:
                    log.info("{id},{name},{status}".format(**r))
            if not response_json["next"]:
                log.info(json.dumps(json_data_list, indent=4))
                return output
            page += 1
            url = self.api.tasks_page(page)
            response = self.session.get(url)
            response.raise_for_status()

        return output

    def get_file_format(self):
        url = self.api.get_file_format()
        response = self.session.get(url)
        response.raise_for_status()
        return response.json()

    def get_dataset_name(self, task_id):
        """Get dataset name by task id"""
        output = self.tasks_list(use_json_output=True)
        dataset_name = [data["name"]
                        for data in output if data["id"] == task_id]

        return [_dataset_name.replace(" ", "_").lower() for _dataset_name in dataset_name]
        

    def tasks_data(self, task_id, resource_type, resources, **kwargs):
        """Add local, remote, or shared files to an existing task."""
        url = self.api.tasks_id_data(task_id)
        data = {}
        files = None
        if resource_type == "local":
            files = {
                "client_files[{}]".format(i): open(f, "rb")
                for i, f in enumerate(resources)
            }
            print(f"files {files}")
            print(f"resources {resources}")
        elif resource_type == "remote":
            data = {"remote_files[{}]".format(
                i): f for i, f in enumerate(resources)}
        elif resource_type == "share":
            data = {"server_files[{}]".format(
                i): f for i, f in enumerate(resources)}
        data["image_quality"] = 70

        # capture additional kwargs
        for flag in [
            "chunk_size",
            "copy_data",
            "image_quality",
            "sorting_method",
            "start_frame",
            "stop_frame",
            "use_cache",
            "use_zip_chunks",
        ]:
            if kwargs.get(flag) is not None:
                data[flag] = kwargs.get(flag)
        if kwargs.get("frame_step") is not None:
            data["frame_filter"] = f"step={kwargs.get('frame_step')}"

        response = self.session.post(url, data=data, files=files)
        response.raise_for_status()
   
    def tasks_frame(self, task_id, frame_ids, outdir="", quality="original", **kwargs):
        for frame_id in frame_ids:
            url = self.api.tasks_id_frame_id(task_id, frame_id, quality)
            response = self.session.get(url)
            response.raise_for_status()
            im = Image.open(BytesIO(response.content))
            mime_type = im.get_format_mimetype() or "image/jpg"
            im_ext = mimetypes.guess_extension(mime_type)
            if im_ext in (".jpe", ".jpeg", None):
                im_ext = ".jpg"

            outfile = "task_{}_frame_{:06d}{}".format(
                task_id, frame_id, im_ext)
            im.save(os.path.join(outdir, outfile))

    def tasks_dump_annotations(
        self, task_id, fileformat, filename=None, extract=False, remove_zip=False, **kwargs
    ):
        url = self.api.tasks_id(task_id)
        response = self.session.get(url)
        response.raise_for_status()
        response_json = response.json()

        if filename is None:
            filename = 'dataset.zip'

        url = self.api.tasks_id_annotations_filename(
            task_id, filename, fileformat
        )
        while True:
            response = self.session.get(url)
            response.raise_for_status()
            log.info("STATUS {}".format(response.status_code))
            if response.status_code == 201:
                break

        response = self.session.get(url + "&action=download")
        response.raise_for_status()

        dataset_name = os.path.join(self.dump_dir, filename)
        dataset_dir = os.path.join(self.dump_dir, filename.split(".")[0])
        # download annotations (.zip)
        with open(filename, "wb") as fp:
            fp.write(response.content)

        if extract:
            with ZipFile(filename, "r") as zip:
                try:
                    os.makedirs(dataset_dir)
                except:
                    shutil.rmtree(dataset_dir)
                zip.extractall(dataset_dir)

        if remove_zip:
            os.remove(dataset_name)

    def tasks_dump(
        self, task_id, fileformat="COCO 1.0", filename='dataset.zip', extract=False, remove_zip=False, to_fiftyone=False, **kwargs
    ):
        url = self.api.tasks_id(task_id)
        response = self.session.get(url)
        response.raise_for_status()

        url = self.api.tasks_id_dataset(task_id, filename, fileformat)
        if to_fiftyone:
            url = self.api.tasks_id_dataset(task_id, filename, "COCO 1.0")

        while True:
            response = self.session.get(url)
            response.raise_for_status()
            log.info("STATUS {}".format(response.status_code))
            if response.status_code == 201:
                break

        url = "".join(
            (url.split("?")[0], "?action=download&", url.split("?")[-1]))
        response = self.session.get(url)
        response.raise_for_status()

        dataset_name = os.path.join(self.dump_dir, filename)
        dataset_dir = os.path.join(self.dump_dir, filename.split(".")[0])
        # download dataset (.zip)
        with open(dataset_name, "wb") as fp:
            fp.write(response.content)

        if extract:
            with ZipFile(dataset_name, "r") as zip:
                try:
                    os.makedirs(dataset_dir)
                except:
                    shutil.rmtree(dataset_dir)
                zip.extractall(dataset_dir)

        if remove_zip:
            os.remove(dataset_name)

        if to_fiftyone:
            convert_format(dataset_dir, format=fileformat)

    def tasks_export(self, task_id, filename, export_verification_period=3, **kwargs):
        export_url = self.api.tasks_id(task_id) + "/backup"

        while True:
            response = self.session.get(export_url)
            response.raise_for_status()
            log.info("STATUS {}".format(response.status_code))
            if response.status_code == 201:
                break
            sleep(export_verification_period)

        response = self.session.get(export_url + "?action=download")
        response.raise_for_status()

        with open(filename, "wb") as fp:
            fp.write(response.content)
        logger_string = "Task {} has been exported sucessfully. ".format(
            task_id
        ) + "to {}".format(os.path.abspath(filename))
        log.info(logger_string)

    def login(self, credentials):
        url = self.api.login
        auth = {"username": credentials[0], "password": credentials[1]}
        headers = {"Content-Type": "application/json"}
        response = self.session.post(url, json.dumps(auth), headers=headers)
        response.raise_for_status()
        if "csrftoken" in response.cookies:
            self.session.headers["X-CSRFToken"] = response.cookies["csrftoken"]



In [4]:
cvat = CVAT(
    username="admin",
    password="supersecret",
    host="https://gslabel-cvat.xscope.ai",
    # dump_dir="/data1/self_driving/cvat/dump/",
    dump_dir= "./"
)


#hannah / Supersecret1
"""
cvat = CVAT(
    username="hannah",
    password="Supersecret1",
    host="http://da.gsenc.com:8081",
    # dump_dir="/data1/self_driving/cvat/dump/",
    dump_dir= "./"
)
"""

'\ncvat = CVAT(\n    username="hannah",\n    password="Supersecret1",\n    host="http://da.gsenc.com:8081",\n    # dump_dir="/data1/self_driving/cvat/dump/",\n    dump_dir= "./"\n)\n'

In [5]:
import sys
import shutil
import os
from pathlib import Path

import torch
from ultralytics import YOLO

import splitfolders

/work/home/jae/miniforge3/envs/cvat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Dataset():
    def __init__(self, images_dirs, dataset_dir="dataset", day=True, night=False):
        self.dataset_dir = dataset_dir
        self.images_dirs = images_dirs
        self.day = day
        self.night = night
        self.missing = []
        self.yaml_path = ""
        
    def get_data(self, labels_dir):
        raw_dataset_dir = self.dataset_dir + "_raw"
        os.makedirs(raw_dataset_dir, exist_ok=True)
        os.makedirs(os.path.join(raw_dataset_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(raw_dataset_dir, "labels"), exist_ok=True)
        for label in os.listdir(labels_dir):
            filename = os.fsdecode(label)[:-4]
            
            image_exists = False

            p = Path("/data1/trash_detection_combined")
            
            for images_dir in [str(x) for x in p.iterdir() if x.is_dir()]:
                # print([x for x in p.iterdir() if x.is_dir()])
                if images_dir[len("/data1/trash_detection_combined") + 1] != "0":
                    continue
                if self.day:
                    # print(os.listdir(os.path.join(images_dir, "01_주간")))
                    if "01_주간" in os.listdir(images_dir) and filename + ".JPG" in os.listdir(os.path.join(images_dir, "01_주간")):
                        shutil.copyfile(os.path.join(images_dir, "01_주간", filename + ".JPG"), os.path.join(raw_dataset_dir, "images", filename + ".JPG"))
                        shutil.copyfile(os.path.join(labels_dir, filename + ".txt"), os.path.join(raw_dataset_dir, "labels", filename + ".txt"))
                        image_exists = True
                if self.night:
                    if "02_야간" in os.listdir(images_dir) and filename + ".JPG" in os.listdir(os.path.join(images_dir, "02_야간")):
                        shutil.copyfile(os.path.join(images_dir, "02_야간", filename + ".JPG"), os.path.join(raw_dataset_dir, "images", filename + ".JPG"))
                        shutil.copyfile(os.path.join(labels_dir, filename + ".txt"), os.path.join(raw_dataset_dir, "labels", filename + ".txt"))
                        image_exists = True
            """
            for images_dir in self.images_dirs:
                p = Path(images_dir)
                # print([x for x in p.iterdir() if x.is_dir()])
                for sub_dir in [str(x) for x in p.iterdir() if x.is_dir()]:
                    # print(sub_dir)
                    if sub_dir[len(images_dir) + 1] != "0":
                        continue
                    if self.day:
                        if "01_주간" in os.listdir(images_dir) and filename + ".JPG" in os.listdir(os.path.join(sub_dir, "01_주간")):
                            # print("COPIED")
                            shutil.copyfile(os.path.join(images_dir, sub_dir, "01_주간", filename + ".JPG"), os.path.join(raw_dataset_dir, "images", filename + ".JPG"))
                            shutil.copyfile(os.path.join(labels_dir, filename + ".txt"), os.path.join(raw_dataset_dir, "labels", filename + ".txt"))
                            image_exists= True
                    if self.night:
                        if "02_야간" in os.listdir(images_dir) and filename + ".JPG" in os.listdir(os.path.join(sub_dir, "02_야간")):
                            shutil.copyfile(os.path.join(images_dir, sub_dir, "02_야간", filename + ".JPG"), os.path.join(raw_dataset_dir, "images", filename + ".JPG"))
                            shutil.copyfile(os.path.join(labels_dir, filename + ".txt"), os.path.join(raw_dataset_dir, "labels", filename + ".txt"))
            """

            if not image_exists:
                self.missing.append(filename)
    
        # splitfolders.ratio(os.path.join(self.dataset_dir, str(task_id)), output=os.path.join(self.dataset_dir, str(task_id)), seed=999, ratio=(0.8, 0.1, 0.1))

    def split(self):
        splitfolders.ratio(self.dataset_dir + "_raw", output=self.dataset_dir, seed=999, ratio=(0.8, 0.1, 0.1))
    
    def make_yaml(self):
        self.yaml_path = os.path.join(self.dataset_dir, "data.yaml")
        with open(os.path.join(self.dataset_dir, "data.yaml"), "w") as f:
            f.write(f"""train: {os.path.join(self.dataset_dir, "train", "images")}
val: {os.path.join(self.dataset_dir, "val", "images")}
test: {os.path.join(self.dataset_dir, "test", "images")}

nc: 5
names: ['TargetArea', 'LineArea', 'Litter', 'Garbage', 'Pile of something']""")

In [8]:
_task_list = cvat.tasks_list()

dataset = Dataset(
    dataset_dir="/work/home/jae/trash_detection/dataset_det",
    images_dirs=["/data1/trash_detection_combined"],
    day=True,
    night=True
)

for task in _task_list:
    task_id = task["id"]
    task_name = task["name"]
    segment_size = task["segment_size"]
    status = task["status"]
    jobs = task["jobs"]
    labels = task["labels"]
    print(f"Task ID: {task_id}, Task Name: {task_name}, Status: {status}, Jobs: {jobs}, Labels: {labels}")
    cvat.tasks_dump_annotations(task_id, fileformat="YOLO 1.1", filename=f"annotations_det/{task_id}.zip", extract=True, remove_zip=True)
    # cvat.tasks_dump_annotations(task_id, fileformat="YOLO ZIP 1.0", filename=f"{task_name}.zip", extract=True, remove_zip=True)
    
    dataset.get_data(f"annotations_det/{task_id}/obj_train_data")

Task ID: 42, Task Name: 0703 images-N, Status: annotation, Jobs: {'count': 0, 'completed': 0, 'url': 'https://gslabel-cvat.xscope.ai/api/jobs?task_id=42'}, Labels: {'url': 'https://gslabel-cvat.xscope.ai/api/labels?task_id=42', 'count': 5}


FileNotFoundError: [Errno 2] No such file or directory: 'annotations_det/42/obj_train_data'

In [ ]:
len(dataset.missing)

In [ ]:
dataset.split()

In [25]:
dataset.make_yaml()

In [9]:
class Model():
    def __init__(self):
        torch.cuda.empty_cache()

        self.model = YOLO("/work/home/jae/cvat/yolov10/ultralytics/cfg/models/v8/yolov8-sch.yaml")

    def train(self, data):
        self.model.train(
            data=data,              # Path to the dataset configuration file
            epochs=100,              # Number of epochs to train for
            time=None,              # Maximum training time
            patience=100,            # Epochs to wait for no observable improvement for early stopping of training
            batch=16,               # Number of images per batch
            imgsz=640,              # Size of input images as integer
            device=3,               # Device to run on, i.e. cuda device=0 
            project="/work/home/jae/trash_detection/results",
            optimizer='auto',       # Optimizer to use, choices=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]
            seed=0,                 # Random seed for reproducibility
            close_mosaic=10,        # Disables mosaic data augmentation in the last N epochs
            freeze=None,            # Freezes the first N layers
            lr0=0.01,               # Initial learning rate 
            lrf=0.01,               # Final learning rate (lr0 * lrf)
            momentum=0.937,         # Momentum factor for SGD or beta1 for Adam optimizers
            weight_decay=0.0005,    # l2 regularization term, penalizing large weights to prevent overfitting
            warmup_epochs=3.0,      # Number of epochs for learning rate warmup
            warmup_momentum=0.8,    # Initial momentum for warmup phase
            warmup_bias_lr=0.1,     # Learning rate for bias parameters during warmup phase
            box=7.5,                # Weight of box loss
            cls=0.5,                # Weight of classification loss
            dfl=1.5,                # Weight of distribution focal loss
            dropout=0.0,            # Use dropout regularization
            cache=False
        )

    def save(self):
        self.model.save("models/yolov8-sch.pt")
        # torch.save(self.model, "models/yolov8-sch.pt")

In [10]:
model = Model()

WARNING ⚠️ no model scale passed. Assuming scale='n'.


In [11]:
dataset.yaml_path

''

In [12]:
model.train("dataset_det/data.yaml")

New https://pypi.org/project/ultralytics/8.2.58 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
engine/trainer: task=detect, mode=train, model=/work/home/jae/cvat/yolov10/ultralytics/cfg/models/v8/yolov8-sch.yaml, data=dataset_det/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=3, workers=8, project=/work/home/jae/trash_detection/results, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, vis

train: Scanning /work/home/jae/trash_detection/dataset_det/train/labels.cache... 1615 images, 89 backgrounds, 0 corrupt: 100%|██████████| 1615/1615 [00:00<?, ?it/s]
val: Scanning /work/home/jae/trash_detection/dataset_det/val/labels.cache... 319 images, 29 backgrounds, 0 corrupt: 100%|██████████| 319/319 [00:00<?, ?it/s]


Plotting labels to /work/home/jae/trash_detection/results/train18/labels.jpg... 


Premature end of JPEG file


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 72 weight(decay=0.0), 86 weight(decay=0.0005), 80 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /work/home/jae/trash_detection/results/train18
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

  0%|          | 0/101 [00:00<?, ?it/s]/work/home/jae/miniforge3/envs/cvat/lib/python3.9/site-packages/torch/autograd/__init__.py:200: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
    100/100      8.05G      1.625      1.485      1.369         32        640:  56%|█████▋    | 57/101 [00:30<00:19,  2.24it/s]Premature end of JPEG file
    100/100      8.05G      1.615      1.473      1.365         30        640:  68%|██████▊   | 69/101 [00:37<00:11,  2.69it/s]Premature end of JPEG file
    100/100      8.05G       1.62      1.475      1.367         35        640:  76%|███████▌  | 77

                   all        319        891      0.606      0.586      0.592      0.342



100 epochs completed in 1.649 hours.
Optimizer stripped from /work/home/jae/trash_detection/results/train18/weights/last.pt, 7.1MB
Optimizer stripped from /work/home/jae/trash_detection/results/train18/weights/best.pt, 7.1MB

Validating /work/home/jae/trash_detection/results/train18/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-sch summary (fused): 235 layers, 3334374 parameters, 0 gradients, 26.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.15it/s]


                   all        319        891      0.635      0.545      0.594      0.342
                Litter        319        320      0.665      0.602       0.66        0.4
               Garbage        319         34      0.621      0.588        0.6      0.372
     Pile of something        319        537       0.62      0.445      0.521      0.255
Speed: 1.1ms preprocess, 4.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /work/home/jae/trash_detection/results/train18


In [127]:
import cv2

img = cv2.imread("/data1/trash_detection_Pyeongtaek_Ilsan/0628 평택/02_야간/G-N-0628-00076.JPG")

model.model.predict(img)


0: 576x640 (no detections), 23.6ms
Speed: 4.3ms preprocess, 23.6ms inference, 0.7ms postprocess per image at shape (1, 3, 576, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'TargetArea', 1: 'LineArea', 2: 'Litter', 3: 'Garbage', 4: 'Pile of something'}
 obb: None
 orig_img: array([[[ 29,   9,   4],
         [ 30,  10,   5],
         [ 30,  10,   5],
         ...,
         [ 83,  49,  33],
         [ 81,  48,  32],
         [ 81,  48,  32]],
 
        [[ 30,  10,   5],
         [ 31,  11,   6],
         [ 31,  11,   6],
         ...,
         [ 84,  50,  34],
         [ 82,  49,  33],
         [ 81,  48,  32]],
 
        [[ 31,  11,   6],
         [ 31,  11,   6],
         [ 31,  11,   6],
         ...,
         [ 83,  49,  33],
         [ 80,  47,  31],
         [ 80,  47,  31]],
 
        ...,
 
        [[158, 146, 144],
         [159, 147, 145],
         [159, 147, 145],
         ...,
         [121, 110, 120],
         [121, 110, 120],
         [121, 110, 120]],
 
        [[159, 147, 145],
         [159,

In [123]:
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    probs = result.probs  # Class probabilities for classification outputs

In [13]:
model.model.val(data="dataset_det/data_switch.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-sch summary (fused): 235 layers, 3334374 parameters, 0 gradients, 26.0 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels.cache... 323 images, 14 backgrounds, 0 corrupt: 100%|██████████| 323/323 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:16<00:00,  1.31it/s]


                   all        323        863      0.684      0.563      0.644      0.377
                Litter        323        276      0.663      0.656      0.699      0.429
               Garbage        323         46      0.732      0.522      0.608      0.402
     Pile of something        323        541      0.658      0.512      0.624      0.301
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /work/home/jae/trash_detection/results/train182


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f24740abfa0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [31]:
model.model.val(data="dataset_det/data_switch.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-chis summary (fused): 243 layers, 3420790 parameters, 0 gradients, 26.0 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels... 323 images, 14 backgrounds, 0 corrupt: 100%|██████████| 323/323 [00:00<00:00, 546.78it/s]

val: New cache created: /work/home/jae/trash_detection/dataset_det/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:15<00:00,  1.39it/s]


                   all        323        863      0.667      0.578      0.636       0.39
                Litter        323        276       0.69      0.659      0.697      0.441
               Garbage        323         46      0.699      0.556      0.617       0.43
     Pile of something        323        541      0.612       0.52      0.594      0.299
Speed: 0.4ms preprocess, 4.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /work/home/jae/trash_detection/results/train172


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd963ec60a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [8]:
dc = YOLO("/work/home/jae/trash_detection/results/dc100/weights/best.pt")

dc.val(data="dataset_det/data_switch.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-dc summary (fused): 221 layers, 4244735 parameters, 0 gradients, 11.0 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels.cache... 135 images, 42 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.17it/s]


                   all        135        243      0.398      0.398      0.402       0.19
                Litter        135         90      0.567      0.625      0.587      0.289
               Garbage        135         11      0.201     0.0909      0.185      0.112
     Pile of something        135        142      0.426      0.479      0.434      0.171
Speed: 1.0ms preprocess, 2.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /work/home/jae/yolov10/yolov10/runs/detect/val8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb22e336700>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [11]:
sch = YOLO("/work/home/jae/trash_detection/results/sch100/weights/best.pt")

sch.val(data="dataset_det/data.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-sch summary (fused): 235 layers, 3334374 parameters, 0 gradients, 26.0 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels.cache... 135 images, 42 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.19it/s]


                   all        135        243      0.453      0.574      0.498      0.258
                Litter        135         90      0.501      0.726      0.677      0.363
               Garbage        135         11      0.473      0.455      0.393      0.228
     Pile of something        135        142      0.385      0.542      0.425      0.183
Speed: 1.1ms preprocess, 5.1ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /work/home/jae/yolov10/yolov10/runs/detect/val8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb43fd95eb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [12]:
schi = YOLO("/work/home/jae/trash_detection/results/schi100/weights/best.pt")

schi.val(data="dataset_det/data.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-chis summary (fused): 243 layers, 3420790 parameters, 0 gradients, 26.0 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels.cache... 135 images, 42 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.16it/s]


                   all        135        243      0.708       0.51       0.55      0.282
                Litter        135         90      0.581       0.71      0.675      0.382
               Garbage        135         11      0.982      0.455      0.514      0.257
     Pile of something        135        142      0.562      0.366       0.46      0.207
Speed: 0.9ms preprocess, 5.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /work/home/jae/yolov10/yolov10/runs/detect/val9


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb43fa427c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [13]:
schi_dc = YOLO("/work/home/jae/trash_detection/results/schi-dc100/weights/best.pt")

schi_dc.val(data="dataset_det/data.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8-schi-dc summary (fused): 299 layers, 4732598 parameters, 0 gradients, 31.1 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels.cache... 135 images, 42 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.17it/s]


                   all        135        243      0.519      0.563      0.504       0.26
                Litter        135         90      0.625      0.767      0.722      0.388
               Garbage        135         11      0.512      0.455       0.37      0.209
     Pile of something        135        142      0.419      0.468      0.421      0.183
Speed: 1.0ms preprocess, 6.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /work/home/jae/yolov10/yolov10/runs/detect/val10


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb410f0ebe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [14]:
vanilla = YOLO("/work/home/jae/trash_detection/results/vanilla100/weights/best.pt")

vanilla.val(data="dataset_det/data.yaml", save=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8 summary (fused): 172 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_det/test/labels.cache... 135 images, 42 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.21it/s]


                   all        135        243      0.576      0.381      0.411      0.192
                Litter        135         90       0.58      0.544      0.562      0.258
               Garbage        135         11      0.651      0.182      0.236      0.146
     Pile of something        135        142      0.497      0.418      0.434      0.172
Speed: 0.7ms preprocess, 1.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /work/home/jae/yolov10/yolov10/runs/detect/val11


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb440183d00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [35]:
dataset.yaml_path

'/work/home/jae/trash_detection/dataset_det/data.yaml'